In [4]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point

In [5]:
with open('data/data_nash.json') as fi:
    credentials = json.load(fi)
    
with open('data/data_nash_secret.json') as fi:
    credentials_secret = json.load(fi)
    
api_key = credentials['api_key']
api_key_secret = credentials_secret['api_key_secret']

In [6]:
endpoint = "https://data.nashville.gov/resource/2u6v-ujjs.json?$limit=50000"

params = {'$where' : "incident_reported between '2021-01-13T00:00:00.000' and '2021-06-30T00:00:00.000'", 
}

In [7]:
response = requests.get(endpoint, params)
response

<Response [200]>

In [8]:
res = response.json()
res = pd.DataFrame(res, columns=['incident_reported', 'offense_description', 'latitude', 'longitude', 'mapped_location', 'zip_code'])
res['date_reported'] = res['incident_reported'].str.split("T", expand=True)[0]
res_trimmed = res.drop(columns='incident_reported')
burglary_reports = res_trimmed[res_trimmed['offense_description'].str.contains('BURGLARY')]
burglary_reports

,offense_description,latitude,longitude,mapped_location,zip_code,date_reported
27,BURGLARY- MOTOR VEHICLE,36.13,-86.7,"{'type': 'Point', 'coordinates': [-86.7, 36.13]}",NaN,2021-01-13
51,BURGLARY- MOTOR VEHICLE,36.07,-86.64,"{'type': 'Point', 'coordinates': [-86.64, 36.07]}",NaN,2021-01-13
56,BURGLARY- MOTOR VEHICLE,36.17,-86.78,"{'type': 'Point', 'coordinates': [-86.78, 36.17]}",NaN,2021-01-13
57,BURGLARY- MOTOR VEHICLE,36.17,-86.78,"{'type': 'Point', 'coordinates': [-86.78, 36.17]}",NaN,2021-01-13
98,BURGLARY- MOTOR VEHICLE,36.21,-86.81,"{'type': 'Point', 'coordinates': [-86.81, 36.21]}",NaN,2021-01-13
...,...,...,...,...,...,...
48822,BURGLARY- MOTOR VEHICLE,36.15,-86.78,"{'type': 'Point', 'coordinates': [-86.78, 36.15]}",NaN,2021-06-29
48859,BURGLARY- MOTOR VEHICLE,36.18,-86.73,"{'type': 'Point', 'coordinates': [-86.73, 36.18]}",NaN,2021-06-29
48866,BURGLARY- AGGRAVATED,36.08,-86.7,"{'type': 'Point', 'coordinates': [-86.7, 36.08]}",NaN,2021-06-29
48881,BURGLARY- MOTOR VEHICLE,36.19,-86.68,"{'type': 'Point', 'coordinates': [-86.68, 36.19]}",NaN,2021-06-29


In [11]:
census_tract = gpd.read_file('data/tl_2019-47_tract/tl_2019_47_tract.shp')
type(census_tract)

geopandas.geodataframe.GeoDataFrame